In [21]:
import pandas as pd

import os
import numpy as np
import psycopg2

from dotenv import load_dotenv
from pathlib import Path

import csv

In [22]:
df = pd.read_csv("../RGBD/table_produits/produits.csv")

In [23]:
replacement = {
    'int64': 'int',
    'object': 'varchar',
    'float64': 'float',
    'datetime64' : 'timestamp',
    'timedelta64[ns]': 'varchar'
}
replacement
 

{'int64': 'int',
 'object': 'varchar',
 'float64': 'float',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [24]:
def generate_column_string(df, replacement):
    return ', '.join("{} {}".format(n, d) for n, d in zip(df.columns, df.dtypes.replace(replacement)))

# Exemple d'utilisation pour chaque DataFrame
col_str_produit = generate_column_string(df, replacement)


# Affichage des résultats
print(col_str_produit)
 


url varchar, nom_produit varchar, img_produit varchar, prix_produit float, contenance varchar, pg_vg varchar, origine varchar, frais varchar, surbooste varchar, saveur varchar, description varchar, brand varchar, gout varchar, info_brand varchar, id_produit int


In [25]:


# Spécifie le chemin vers votre fichier .env
env_path = Path('../.env')
load_dotenv(dotenv_path=env_path)

# Récupère les paramètres de connexion depuis le fichier .env
conn_params = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'connect_timeout': os.getenv('DB_TIMEOUT'),
    'sslmode': os.getenv('DB_SSLMODE')
}

# Connexion à la base de données PostgreSQL
try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    print("Opened database successfully")
except Exception as e:
    print(f"Error: {e}")
    

Opened database successfully


In [26]:
# drop tables with same name
cursor.execute("DROP TABLE IF EXISTS produits;")
 

In [27]:
# Créer la table produits avec les colonnes spécifiées
cursor.execute(f"""
CREATE TABLE produits (
    url VARCHAR,
    nom_produit VARCHAR,
    img_produit VARCHAR,
    prix_produit FLOAT,
    contenance VARCHAR,
    pg_vg VARCHAR,
    origine VARCHAR,
    frais VARCHAR,
    surbooste VARCHAR,
    saveur VARCHAR,
    description VARCHAR,
    brand VARCHAR,
    gout VARCHAR,
    info_brand VARCHAR,
    id_produit INT
);
""")
 

In [28]:
my_file = open('../RGBD/table_produits/produits.csv')
print('File opened in memory')
 

File opened in memory


In [29]:
# SQL pour insérer les données dans la table
SQL_STATEMENT_PRODUITS = """
COPY produits FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
    QUOTE '"'
"""

cursor.copy_expert(sql=SQL_STATEMENT_PRODUITS, file=my_file)
print('file copie to db')
 

file copie to db


In [30]:
# Exécuter la copie des données dans la table
cursor.copy_expert(sql=SQL_STATEMENT_PRODUITS, file=my_file)
print('Fichier copié dans la base de données.')

# Fermer la connexion
conn.commit()
cursor.close()
conn.close()
print("Connexion fermée.") 
 

Fichier copié dans la base de données.
Connexion fermée.
